# Tutorial 1 for Python

## Solve Dantzig's Transport Problem using the *ix modeling platform* (ixmp)

<img style="float: right; height: 80px;" src="_static/python.png">

This problem finds a least cost shipping schedule that meets requirements at markets and supplies at factories.

Dantzig, G B, Chapter 3.3. In Linear Programming and Extensions.
Princeton University Press, Princeton, New Jersey, 1963.

This formulation is described in detail in:
Rosenthal, R E, Chapter 2: A GAMS Tutorial. 
In GAMS: A User's Guide. The Scientific Press, Redwood City, California, 1988.

see http://www.gams.com/mccarl/trnsport.gms

### Launching the platform and initializing a new datastructure

This datastructure will be used to store all data required to solve Dantzig's transport problem as well as the solution after solving it in GAMS.

In [ ]:
# load required packages 
import pandas as pd
import ixmp as ix

In [ ]:
# launch the IX modeling platform using the local default database                                                                                                                       
mp = ix.Platform(dbtype='HSQLDB')

In [ ]:
# details for creating a new scenario in the IX modeling platform                                                                                                                                                     
model = "canning problem"
scenario = "standard"
annot = "Dantzig's transportation problem for illustration and testing"   

# initialize a new Scenario instance
# the parameter version='new' indicates that this is a new datastructure
scen = mp.Scenario(model, scenario, version='new', annotation=annot)

### Defining the sets in the datastructure

In [ ]:
# define the sets of locations of canning plants and markets                                                                                                                             
scen.init_set("i")                                                                                                                                                                      
scen.add_set("i", ["seattle", "san-diego"])                                                                                                                                             
scen.init_set("j")                                                                                                                                                                      
scen.add_set("j", ["new-york", "chicago", "topeka"])                                                                                                                                    

In [ ]:
# display the set 'i'
scen.set('i')

### Defining parameters in the datastructure

In [ ]:
# capacity of plant i in cases                                                                                                                                                           
# add parameter elements one-by-one (string and value)                                                                                                                                   
scen.init_par("a", idx_sets="i")
scen.add_par("a", "seattle", 350, "cases")
scen.add_par("a", "san-diego", 600, "cases")

# demand at market j in cases                                                                                                                                                            
# add parameter elements as dataframe (with index names)                                                                                                                                 
scen.init_par("b", idx_sets="j")
b_data = [
    {'j': "new-york", 'value': 325, 'unit': "cases"},
    {'j': "chicago",  'value': 300, 'unit': "cases"},
    {'j': "topeka",   'value': 275, 'unit': "cases"}
]
b = pd.DataFrame(b_data)
scen.add_par("b", b)

In [ ]:
ds.par('b')

In [ ]:
# distance in thousands of miles                                                                                                                                                         
scen.init_par("d", idx_sets=["i", "j"])
# add more parameter elements as dataframe by index names                                                                                                                                
d_data = [
    {'i': "seattle", 'j': "new-york", 'value': 2.5, 'unit': "km"},
    {'i': "seattle", 'j': "chicago", 'value': 1.7, 'unit': "km"},
    {'i': "seattle", 'j': "topeka", 'value': 1.8, 'unit': "km"},
    {'i': "san-diego", 'j': "new-york", 'value': 2.5, 'unit': "km"},
]
d = pd.DataFrame(d_data)
scen.add_par("d", d)

# add other parameter elements as key list, value, unit
scen.add_par("d", ["san-diego", "chicago"], 1.8, "km")
scen.add_par("d", ["san-diego", "topeka"], 1.4, "km")

In [ ]:
# cost per case per 1000 miles                                                                                                                                                           
# initialize scalar with a value and a unit (and optionally a comment)                                                                                                                   
scen.init_scalar("f", 90.0, "USD/km")

### Saving the datastructure to the IX Modeling Platform database

In [ ]:
# commit new datastructure to the database
# no changes can then be made to the datastructure until a check-out is performed
comment = "importing Dantzig's transport problem for illustration"
comment += " and testing of the Python interface using a generic datastructure"                                                                                                                                                                            
scen.commit(comment)      

# set this new datastructure as the default version for the model/scenario name
scen.set_as_default()

### Defining variables and equations in the datastructure

The levels and marginals of these variables and equations will be imported to the datastructure when reading the gdx solution file.

In [ ]:
# perform a check_out to make further changes
scen.check_out()

# initialize the decision variables and equations
scen.init_var("z", None, None)
scen.init_var("x", idx_sets=["i", "j"])
scen.init_equ("demand", idx_sets=["j"])

# save changes to database
change_comment = "inialize the model variables and equations"
scen.commit(change_comment)

### Solve the model

The ``solve()`` function exports the datastructure to a GAMS gdx file, executes GAMS, and then imports the solution from an output GAMS gdx file to the database.

For the model equations and the GAMS workflow (reading the data from gdx, solving the model, writing the results to gdx), see ``transport_ixmp.gms``.

In [ ]:
scen.solve(model='transport_ixmp', case='transport_standard')

### Display and analyze the results

In [ ]:
# display the objective value of the solution
scen.var("z")

In [ ]:
# display the quantities transported from canning plants to demand locations
scen.var("x")

In [ ]:
# display the quantities and marginals (=shadow prices) of the demand balance constraints
scen.equ("demand")

### Close the database connection of the IX Modeling Platform

This is recommended when working with the local file-based database, i.e., dbtype='HSQLDB'. This command closes the database files and removes temporary data, so that the database files can be copied to a different folder or drive.

In [ ]:
# close the connection of the platform instance to the database
scen.close_db()